In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import tensorflow as tf
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [2]:
ds = pd.read_csv("/kaggle/input/digit-recognizer/train.csv")
test = pd.read_csv("/kaggle/input/digit-recognizer/test.csv")

train, val = train_test_split(ds, test_size=0.2)

X_train = (train.iloc[:,1:].values).astype('float32') # all pixel values
y_train = train.iloc[:,0].values.astype('int32') # only labels i.e targets digits

X_test = test.values.astype('float32')


In [3]:
X_train = X_train.reshape(len(X_train), 28, 28, 1) # reshape to (num_imgs, width, height, channel)
X_test = X_test.reshape(len(X_test), 28, 28, 1) # reshape to (num_imgs, width, height, channel)

In [4]:
# normalize
X_train = X_train / 255.0
X_test = X_test / 255.0

In [5]:
y_train = tf.keras.utils.to_categorical(y_train, num_classes = 10)


In [6]:
# plot image i'th
def display(i, dataset):
    plt.imshow(dataset[i][:,:,0])    
display(1234, X_train)

In [7]:
EPOCHS = 3
BATCH_SIZE = 32
LEARNING_RATE = 1e-4

In [8]:
def build_model():
    model = tf.keras.Sequential(
    [
        tf.keras.layers.Conv2D(32, 5, 1, padding='same', activation='relu', input_shape=(28,28,1)),
        tf.keras.layers.MaxPooling2D(2),
        tf.keras.layers.Conv2D(64, 5, 1, padding='same', activation='relu'),
        tf.keras.layers.MaxPooling2D(2),
        tf.keras.layers.Flatten(),
        tf.keras.layers.Dense(256, activation='relu'),
        tf.keras.layers.Dense(10, activation='softmax'),
    ])
    return model
model = build_model()
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=LEARNING_RATE),
              loss=tf.keras.losses.CategoricalCrossentropy(),
              metrics=['accuracy'])

In [9]:
history = model.fit(X_train, y_train, batch_size=BATCH_SIZE, epochs=EPOCHS)

In [10]:
# predict results
results = model.predict(X_test)

# select the indix with the maximum probability
results = np.argmax(results,axis = 1)

results = pd.Series(results,name="Label")

In [15]:
submissions=pd.DataFrame({"ImageId": list(range(1,len(results)+1)),
                         "Label": results})
submissions.to_csv("/kaggle/working/submission.csv", index=False, header=True)